In [12]:
import cv2
import cv2.aruco as aruco
import numpy as np

# 1) 카메라 내부 파라미터 (예시)
#   - 실제 사용 시, 캘리브레이션으로 획득한 matrix, distCoeffs를 넣으세요.
cameraMatrix = np.array([[1400, 0,    640],
                         [0,    1400, 360],
                         [0,    0,    1]], dtype=np.float32)
distCoeffs = np.zeros((5,), dtype=np.float32)  # 왜곡 계수(예: 없는 경우)

# 2) 마커 실제 한 변 길이 (단위 m)
marker_length = 0.18

# 3) ArUco 사전, 파라미터 생성 (DetectorParameters_create() 대신 구버전 스타일)
# 구버전 OpenCV에서 주로 지원되던 함수
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

detector_params = aruco.DetectorParameters()  # 예전 방식

# 4) 월드 좌표에서 마커 두 개의 위치(예시)
#    만약 마커 ID 0,1이라 가정
marker_world_pos = {
    0: np.array([0.0,  0.0, 0.0]),  # marker ID=0 의 (X,Y,Z)
    1: np.array([0.3,  0.0, 0.0])   # marker ID=1 의 (X,Y,Z) -> 30cm 옆
}

def compute_camera_pos(marker_id, rvec, tvec, marker_world):
    """
    단일 마커의 pose (rvec, tvec) + 해당 마커의 월드 좌표 -> 카메라 월드 위치 추정
    단순히 '마커가 world orientation=I' 라고 가정하여,
      camera_in_marker = -R^T * t
      camera_world = marker_world + camera_in_marker
    """
    R, _ = cv2.Rodrigues(rvec)        # (3,3)
    t = tvec.reshape(3,1)             # (3,1)
    cam_in_marker = -R.T @ t          # (3,1)
    return marker_world.reshape(3,1) + cam_in_marker

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("카메라를 열 수 없습니다.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패!")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 5) ArUco detect
    corners, ids, rejected = aruco.detectMarkers(
        gray, aruco_dict, parameters=detector_params
    )

    if ids is not None:
        # 마커 그리기
        aruco.drawDetectedMarkers(frame, corners, ids)
        
        # 6) Pose 추정
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(
            corners, marker_length, cameraMatrix, distCoeffs
        )
        for i, id_ in enumerate(ids):
            # axis 그리기
            #aruco.drawAxis(frame, cameraMatrix, distCoeffs, rvecs[i], tvecs[i], 0.03)
            # 텍스트
            c = corners[i][0][0]
            cv2.putText(frame, f"ID {id_[0]}", (int(c[0]), int(c[1]-10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0),2)

        # 7) 두 개 마커(ID=0,1 둘 다 검출 시), 카메라 위치 추정
        #    실제로는 ids가 [0,1,...] 등 여러개.
        #    여기서는 간단히 '0번과1번'만 찾는다
        found_ids = ids.flatten()
        if 0 in found_ids and 1 in found_ids:
            idx0 = np.where(found_ids==0)[0][0]
            idx1 = np.where(found_ids==1)[0][0]

            r0, t0 = rvecs[idx0], tvecs[idx0]
            r1, t1 = rvecs[idx1], tvecs[idx1]

            # marker0_cam, marker1_cam
            cam_pos0 = compute_camera_pos(0, r0, t0, marker_world_pos[0])
            cam_pos1 = compute_camera_pos(1, r1, t1, marker_world_pos[1])

            # 단순히 평균
            camera_world_pos = 0.5*(cam_pos0 + cam_pos1)
            cx, cy, cz = camera_world_pos.ravel()
            print(f"Camera pos ~ ({cx:.2f}, {cy:.2f}, {cz:.2f})")

    # 화면 표시
    cv2.imshow("Aruco 2 Marker Pose", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()


Camera pos ~ (0.06, 0.41, 2.94)
Camera pos ~ (-0.01, 0.44, 2.93)
Camera pos ~ (0.88, 0.08, 2.94)
Camera pos ~ (0.13, 0.49, 2.97)
Camera pos ~ (0.76, 0.23, 2.95)
Camera pos ~ (0.09, 0.53, 3.00)
Camera pos ~ (0.65, 0.18, 2.98)
Camera pos ~ (0.13, 0.50, 3.04)
Camera pos ~ (0.71, 0.16, 3.02)
Camera pos ~ (0.63, 0.24, 3.01)
Camera pos ~ (0.49, 0.05, 3.07)
Camera pos ~ (0.51, 0.28, 3.03)
Camera pos ~ (0.61, 0.24, 3.05)
Camera pos ~ (0.64, 0.26, 3.03)
Camera pos ~ (0.56, 0.23, 3.08)
Camera pos ~ (0.14, 0.70, 3.06)
Camera pos ~ (0.10, 0.60, 3.09)
Camera pos ~ (-0.09, 0.44, 3.10)
Camera pos ~ (0.02, 0.46, 3.15)
Camera pos ~ (0.19, 0.52, 3.15)
Camera pos ~ (0.06, 0.66, 3.10)
Camera pos ~ (-0.10, 0.60, 3.10)
Camera pos ~ (0.07, 0.64, 3.12)
Camera pos ~ (0.14, 0.57, 3.14)
Camera pos ~ (0.12, 0.70, 3.12)
Camera pos ~ (0.03, 0.74, 3.09)
Camera pos ~ (0.15, 0.74, 3.11)
Camera pos ~ (0.13, 0.80, 3.10)
Camera pos ~ (0.11, 0.82, 3.08)
Camera pos ~ (0.08, 0.77, 3.11)
Camera pos ~ (0.08, 0.78, 3.11)
Camer